<a id='top'></a><a name='top'></a>
# Building Machine Learning Pipelines

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/gbih/ml/blob/main/tfx/nb_01_interactive_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

1. [Introduction](#1.0)
    * 1.1 [Data](#1.1)
    * 1.2 [Problem](#1.2)
    * 1.3 [Model](#1.3)
    * 1.4 [Pipeline Orchestration](#1.4)
2. [Setup](#2.0)
    * 2.1 [GPU check](#2.1)
    * 2.2 [Setup project structure](#2.2)
    * 2.3 [Download dataset](#2.3)
    * 2.4 [Quick check of missing data and encoding format](#2.4)
3. [Create InteractiveContext](#3.0)
4. [Data ingestion](#4.0)
   * [4.1. ExampleGen](#4.1)
5. [Data validation](#5.0)
    * [5.1. StatisticsGen](#5.1)
    * [5.2. SchemaGen](#5.2)
    * [5.3. ExampleValidator](#5.3)
6. [Data preprocessing](#6.0)
    * [6.1. Transform](#6.1)
        - [6.1.1 tfx.components.Transform](#6.1.1)
7. [Model training](#7.0)
    * [7.1. Trainer](#7.1)
8. [Model analysis and validation](#8.0)
    * [8.1. Resolver](#8.1)
    * [8.2. Evaluator](#8.2)
    * [8.3. Pusher](#8.3)
9. [Testing SavedModels](#9.0)
    * [9.1 Testing inference with tf.keras.models.load_model](#9.1)
    * [9.2 saved_model_cli tool](#9.2)
        - [9.2.1 Testing inference with saved_model_cli run](#9.2.1)
10. [Model deployment](#10.0)
    * [10.1 TensorFlow Model Server](#10.1)
        - [10.1.1 Colab TF ModelServer](#10.1.1)
        - [10.1.2 Docker TF ModelServer](#10.1.2)
        - [10.1.3 Start Server](#10.1.3)
        - [10.1.4 Check Server and Logs](#10.1.4)
11. [TensorFlow Model Server - RESTful APIs](#11.0)
    * [11.1 Status API](#11.1)
    * [11.2 Metadata API](#11.2)
    * [11.3 Predict API (simple client)](#11.3)
    * [11.4 Predict API (complex client)](#11.4)
12. [Miscellaneous](#12.0)
13. [Quit TensorFlow Model Server](#13.0)

---
## Environment

```
Python version       : 3.7.15

tfx                      : 1.10.0
tensorflow               : 2.9.2
tensorflow_transform     : 1.10.1
tensorflow_model_analysis: 0.41.1
```

---
<a name='1.0'></a><a id='1.0'></a>
# 1. Introduction
<a href="#top">[back to top]</a>


<a name="1.1"></a>
## 1.1 Data
<a href="#top">[back to top]</a>

This example project uses open source data from the [US Consumer Finance Protection Bureau](https://www.consumerfinance.gov/data-research/consumer-complaints/). This specific dataset is a collection of consumer complaints about financial products in the United States, and contains a mixture of structured data (categorical/numeric data) and unstructured text data.


<a name="1.2"></a>
## 1.2 Problem
<a href="#top">[back to top]</a>

The goal is to predict whether a consumer disputed a complaint about a financial product. We can organize this as a binary classification problem.


<a name="1.3"></a>
## 1.3 Model
<a href="#top">[back to top]</a>

**Model Type**

In their book, Hapke & Nelson [use a model patterned after the Wide-and-Deep architecture](https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines/blob/main/components/module.py), with the addition of the Universal Sentence Encoder from TensorFlow Hub. The latter is used to encode the free-text feature (customer complaint text data).

We can try sketching out in pseudo-code a rough Keras model. This model will have two parts, binary classification and wide-and-deep. To enable the latter, we need to organize the model using the Keras functional API. 

Since this is a classification problem, we want to measure the performance of our model, whose output is a probability value between 0 and 1. The loss of 0 indicates a perfect model. We use the BinaryCrossentropy loss function.


```python
deep = ...
wide = ...
both = tf.keras.layers.concatenate([deep, wide]) 

# built-up inputs from dataset features
inputs = build_inputs_fn() 
# binary output for label
output = tf.keras.layers.Dense(1, activation="sigmoid")(both) 
keras_model = tf.keras.models.Model(inputs, output)

keras_model.compile(
    optimizer="adam"
    loss="binary_crossentropy",
    metrics=["BinaryAccuracy"],
)
```


**Model Features and Labels**

* FEATURES
    1. Consumer complaint narrative
    2. Issue that the consumer complained about
    3. Sub-issue
    4. Financial product
    5. Subproduct
    6. State
    7. Timely response or not
    8. Zip code
* LABEL
    1. Consumer disputed or not


<a name="1.4"></a>
## 1.4 Pipeline Orchestration
<a href="#top">[back to top]</a>

**TFX Components**

We will use the following components, in this order:

1. Data ingestion:
    * `tfx.components.CsvExampleGen`
2. Data validation 
    * `tfx.components.StatisticsGen`
    * `tfx.components.SchemaGen`
    * `tfx.components.ExampleValidator`
3. Data preprocessing
    * `tfx.components.Transform`
4. Model training
    * `tfx.components.Trainer`
5. Model analysis and validation
    * `tfx.components.Resolver`
    * `tfx.components.Evaluator`
    * `tfx.components.Pusher`

Some TFX components that we will not use here include `InfraValidator`, `Tuner`, `BulkInferrer`. 

During the last step, we deploy the blessed-model artifact to TensorFlow Serving.

Resources:
* [Module: tfx.v1.components](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components)

**Orchestration Engine**

The flow of data and artifacts between components, and the operation of the individual components themselves, comprise our pipeline. 

There are several options to manage end-to-end orchestration of this pipeline.

For production use, we can use Apache Airflow, Apache Beam, Kubeflow Pipelines, etc. These options are appropriate for production-level TFX pipelines. In this setting, the orchestration engine automatically monitors and executes components.

For pipeline experimentation, it can be easier to use notebooks. Here, the user is the orchestrator, and manually runs the individual notebook cells. For this, we use the `InteractiveContext` tool, which manages component execution and state in the notebook.

Resources:

* [Run TFX in Google Colab blog post](https://blog.tensorflow.org/2019/11/introducing-tfx-interactive-notebook.html)

---
<a name='2.0'></a><a id='2.0'></a>
# 2. Setup
<a href="#top">[back to top]</a>

In [1]:
import sys
import subprocess 

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running on Colab. Installing TFX.")
    !pip install tfx &> /dev/null
    !apt-get install tree &> /dev/null
    print("You need to restart the Colab runtime after installing TFX:")
else:
    print("Running locally.")

Running locally.


<a name='2.1'></a><a id='2.1'></a>
## 2.1 GPU check
<a href="#top">[back to top]</a>

To enable GPU on Colab for this notebook:

-  Edit -> Notebook Settings
-  Hardware Accelerator drop-down -> GPU

Confirm GPU connction with tensorflow:

In [ ]:
# If need to start with a clean project:
# !rm -fr tfx_data_01
# !rm -fr tfx_data_src_01
# !rm -fr tfx_output_01
# !rm -fr tmp

In [2]:
import sys
import subprocess 

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('GPU device not found')
    print('Found GPU at: {}'.format(device_name))

In [3]:
# stdlib imports
import base64
import glob
import os
import pathlib
from pathlib import Path
import pprint
import requests
import shlex
import sys
import time

# third party imports
import tensorflow as tf
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

if IN_COLAB:
    !pip install tensorflow-model-analysis -q
    !pip install tensorflow-transform -q
    !pip install cchardet -q
    !pip install watermark -q

import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import cchardet as cchardet
from watermark import watermark

if not IN_COLAB:
    import docker 

tf.get_logger().propagate = False
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel('INFO') # DEBUG, INFO, WARN, ERROR, or FATAL

pp = pprint.PrettyPrinter(indent=4)

def HR():
    print('-'*40)

# For debugging, provides version & hardware info
print("VERSION CHECK:\n")
print(watermark(packages="tfx,tensorflow,tensorflow_transform,tensorflow_model_analysis", python=True,machine=True,hostname=True))
HR()

print("Finished imports")

2022-11-06 22:22:19.556099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


VERSION CHECK:

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.34.0

tfx                      : 1.9.1
tensorflow               : 2.10.0
tensorflow_transform     : 1.9.0
tensorflow_model_analysis: 0.40.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.3)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Hostname: Georges-MacBook-Air.local

----------------------------------------
Finished imports


<a name="2.2"></a>
## 2.2 Setup project structure
<a href="#top">[back to top]</a>

Organize two main assets, pipeline-output and source-data,  tfx_output_01 and tfx_data_01

Notes:

* pathlib vs os.path: pathlib.Path is newer and can replace instead of os.path. However, many of the TFX components require casting the PosixPath to bytes or unicode, which we can do via .as_posix(). So, which is more convenient is debatable.


In [4]:
# 1. Setup pathways for TFX artifact and metadata output
OUTPUT_ROOT = 'tfx_output_01'
PIPELINE_NAME = 'customer_complaints'
PIPELINE_ROOT = Path(OUTPUT_ROOT, 'tfx_pipeline_output', PIPELINE_NAME).as_posix()
METADATA_PATH = Path(OUTPUT_ROOT, 'tfx_metadata', PIPELINE_NAME, 'metadata.db').as_posix()

# Need absolute path to setup TensorFlow Serving
SERVING_MODEL_DIR = Path(OUTPUT_ROOT, 'tfx_serving_model', PIPELINE_NAME).resolve().as_posix()

print(
f"""OUTPUT_ROOT: {OUTPUT_ROOT}
PIPELINE_NAME: {PIPELINE_NAME}
PIPELINE_ROOT: {PIPELINE_ROOT}
METADATA_PATH: {METADATA_PATH}
""")

# 2. Setup pathway for downloaded project data
DATA_DL_ROOT = Path('tfx_data_src_01', PIPELINE_NAME).as_posix()
DATA_DL_URL = 'http://bit.ly/building-ml-pipelines-dataset'
DATA_DL_SRC = Path(DATA_DL_ROOT).as_posix()
DATA_DL_FILE = "data_dl.csv"
DATA_DL_PATH = Path(DATA_DL_ROOT, DATA_DL_FILE).as_posix()

print(
f"""DATA_DL_ROOT: {DATA_DL_ROOT}
DATA_DL_URL: {DATA_DL_URL}
DATA_DL_SRC: {DATA_DL_SRC}
DATA_DL_FILE: {DATA_DL_FILE}
DATA_DL_PATH: {DATA_DL_PATH}
""")

# 3. Setup pathway for project data
DATA_ROOT = Path('tfx_data_01', PIPELINE_NAME).as_posix()
DATA_SRC = Path(DATA_ROOT).as_posix()
DATA_FILE = "data.csv"
DATA_PATH = Path(DATA_ROOT, DATA_FILE).as_posix()

print(
f"""DATA_ROOT: {DATA_ROOT}
DATA_SRC: {DATA_SRC}
DATA_FILE: {DATA_FILE}
DATA_PATH: {DATA_PATH}

SERVING_MODEL_DIR: {SERVING_MODEL_DIR}
""")

OUTPUT_ROOT: tfx_output_01
PIPELINE_NAME: customer_complaints
PIPELINE_ROOT: tfx_output_01/tfx_pipeline_output/customer_complaints
METADATA_PATH: tfx_output_01/tfx_metadata/customer_complaints/metadata.db

DATA_DL_ROOT: tfx_data_src_01/customer_complaints
DATA_DL_URL: http://bit.ly/building-ml-pipelines-dataset
DATA_DL_SRC: tfx_data_src_01/customer_complaints
DATA_DL_FILE: data_dl.csv
DATA_DL_PATH: tfx_data_src_01/customer_complaints/data_dl.csv

DATA_ROOT: tfx_data_01/customer_complaints
DATA_SRC: tfx_data_01/customer_complaints
DATA_FILE: data.csv
DATA_PATH: tfx_data_01/customer_complaints/data.csv

SERVING_MODEL_DIR: /Users/gb/Desktop/TFX-CUSTOM-COMPONENTS/book/tfx_output_01/tfx_serving_model/customer_complaints



<a name="2.3"></a>
## 2.3 Download dataset
<a href="#top">[back to top]</a>

In [ ]:
# try:
#     Path(DATA_ROOT).mkdir(parents=True, exist_ok=True)
#     path_to_downloaded_file = tf.keras.utils.get_file(
#         fname=DATA_FILE,
#         origin=DATA_URL,
#         cache_dir=DATA_SRC,
#         cache_subdir=""
#     )    
#     print(f"Downloaded {DATA_URL} to {DATA_SRC}")
# except Exception as e:
#     print(f"Error: {e}")

try:
    Path(DATA_DL_ROOT).mkdir(parents=True, exist_ok=True)
    path_to_downloaded_file = tf.keras.utils.get_file(
        fname=DATA_DL_FILE,
        origin=DATA_DL_URL,
        cache_dir=DATA_DL_SRC,
        cache_subdir=""
    )    
    print(f"Downloaded {DATA_DL_URL} to {DATA_DL_SRC}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# # Check new output in data directory
# !tree {DATA_ROOT}

# Check new output in data directory
!tree {DATA_DL_ROOT}

### 2.3.1 Create smaller dataset for memory reasons

In [ ]:
!du -h {DATA_DL_PATH}

In [ ]:
os.makedirs(DATA_SRC, exist_ok=True) 
!du -h {DATA_SRC}

---
FILE_LENGTH=2500, get this error:

RuntimeError: tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[512,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill] [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractPredictions/Inference']

In [ ]:
# Only use the first n-lines of the CSV file, for now
FILE_LENGTH=1000

!sed {FILE_LENGTH}q {DATA_DL_PATH} > {DATA_PATH}

In [ ]:
!du -h {DATA_SRC}

In [ ]:
!ls -l {DATA_SRC}

<a name="2.4"></a>
## 2.4 Quick examination of data and encoding format
<a href="#top">[back to top]</a>

Although Pandas is not a part of the TFX pipeline per se, it is a very useful part of the workflow in the early stages of a project. It is very convenient and flexible in terms of quickly getting a feel for the data.

<a name="2.4.1"></a>
### 2.4.1 Examining the data
<a href="#top">[back to top]</a>

Use Pandas to check for missing values, types, label distribution skew

---
**Checking for null values**

In [ ]:
import pandas as pd

df = pd.read_csv(DATA_PATH)
print(df.info())
print()
HR()
print("Sum of isnull() per feature:\n")
print(df.isnull().sum())
HR()
print(len(df))

---
**Checking feature types**

In [ ]:
types = list(df.dtypes.unique())
print(types)
HR()

for x in types:
    print(x)
    print(list(df.select_dtypes([x]).columns))
    HR()

---
**Checking labels**

* `consumer_disputed == 1`: Consumer disputed a complaint about a financial product
* `consumer_disputed == 0`: Consumer did not dispute a complaint about a financial product

In [ ]:
# consumer_disputed == 1: Consumer disputed a complaint about a financial product

df.loc[(df['consumer_disputed'] == 1)].head(2).T

In [ ]:
# consumer_disputed == 0: Consumer did not dispute a complaint about a financial product

df.loc[(df['consumer_disputed'] == 0)].head(2).T

---

**Checking label distribution skew**

We can see that the majority of labels are non-disputed (around 75%).

In [ ]:
df['consumer_disputed'].value_counts(normalize=True)

<a name="2.4.2"></a>
### 2.4.2 Encoding format
<a href="#top">[back to top]</a>

Sometimes it can be useful to confirm the encoding of the dataset.

In [ ]:
# https://www.kaggle.com/code/rtatman/automatically-detecting-character-encodings/notebook
# https://chardet.readthedocs.io/en/latest/usage.html
def check_encoding(dir, line_n=5):
    for filename in glob.glob(f'{dir}/*'):
        
        print(f"file:\t\t{filename}")
        
        lines_all = !(cat {filename} | wc -l)
        lines_all = int((lines_all[0].strip()))
        print(f"total lines:\t{lines_all:,}")

        # sanity check to not exceep length of file
        if (line_n > lines_all):
            line_n = lines_all
        
        with open(filename, 'rb') as rawdata:
            first_n_lines = [next(rawdata) for _ in range(line_n)]
            result = cchardet.detect(first_n_lines[0])
        print(f"{line_n:,} lines: \t{result} encoding")


# If all characters in sequence is ascii symbols, chardet outputs ascii
check_encoding(DATA_ROOT, 1000)

---
<a name='3.0'></a><a id='3.0'></a>
# 3. Create InteractiveContext
<a href="#top">[back to top]</a>

Create a pipeline for interactive TFX notebook development.

For experimentation with TFX pipeline (such as here), it is easier to use notebooks. In this experimentation setting, the user manually running the notebook cells is the orchestrator. We use the `InteractiveContext` tool, which manages component execution and state in the notebook.

Basic concepts:

* To create an InteractiveContext, we call `context = InteractiveContext()`. This  manages component execution and state in the notebook. Default parameters use a temporary directory with an ephemeral ML Metadata database instance. We can also specify our own pipeline root or database by passing  the optional properties `pipeline_root` and `metadata_connection_config`
 
```python
context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    pipeline_name=PIPELINE_NAME,
    metadata_connection_config=tfx_metadata
)
```

* To run the component, we call `context.run()` and run that cell.

```python
context.run(statistics_gen)
```

* To use built-in TFX visualization tools, we call `context.show()`. For example, to  review statistics with a built-in TFX visualization:

```python
context.show(statistics_gen.outputs['statistics'])
```



Note:

If we use sqlite format for `metadata_connection_config`, we can easily examine the .db file with an external tool such as "DB Browser for SQLite Version", etc

Resources:

* https://github.com/tensorflow/tfx/blob/0422c4e4017fcc8a671ded67d8954b4a0f11edf4/tfx/orchestration/experimental/interactive/interactive_context.py#L72


In [ ]:
tfx_metadata=tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH 
)

# At this step, this only creates the pathway, this does not actually create the data-store yet.
context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    pipeline_name=PIPELINE_NAME,
    metadata_connection_config=tfx_metadata
)

In [ ]:
# Check new artifacts in pipeline directory
!tree {OUTPUT_ROOT}

---
<a name='4.0'></a><a id='4.0'></a>
# 4. Data ingestion 
<a href="#top">[back to top]</a>

<a name='4.1'><a id='4.1'></a>
## 4.1 ExampleGen
<a href="#top">[back to top]</a>

Create a TFX CsvExampleGen component.

The `tfx.components.CsvExampleGen` component takes csv data, and generates train and eval examples for downstream components.This component encodes column values to tf.Example int/float/byte feature. 

Resources: 

* https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/CsvExampleGen

In [ ]:
# The trainer ingests data into TFX pipeline. This make take a lot of time.
example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT
)

example_gen.__dict__

In [ ]:
# Run a given TFX component in the interactive context.
context.run(example_gen, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline directory
!tree {PIPELINE_ROOT}

In [ ]:
# Show the given object in an IPython notebook display.
context.show(example_gen)

In [ ]:
# Examine output artifacts of ExampleGen, training and evaluation examples
# These are the component's output channel dict.
artifact = example_gen.outputs['examples'].get()[0]

print(artifact.split_names)
print(artifact.uri)

In [ ]:
# Take a look at training examples

# Get the URI of the training example output artifact, which is a dir
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')
print(f"train_uri:\n{train_uri}")
HR()

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
print(f"tfrecord_filenames:\n{tfrecord_filenames}")
HR()

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
print(f"dataset:\n{dataset}")
HR()

# Iterate over the first n records and decode them
print("Examine records:\n")
for i, tfrecord in enumerate(dataset.take(2)):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    print(f"=== Record {i} ===\n")
    print(example)
    HR()

---
<a id='5.0'></a><a name='5.0'></a>
# 5. Data validation
<a href="#top">[back to top]</a>

<a name='5.1'></a><a id='5.1'></a>
## 5.1 StatisticsGen
<a href="#top">[back to top]</a>

The StatisticsGen component generates features statistics and random samples over both training and serving data, which can be used for visualization and validation. StatisticsGen uses Apache Beam and approximate algorithms to scale to large datasets.

Resources:

* https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/StatisticsGen

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)

statistics_gen.__dict__

In [ ]:
context.run(statistics_gen, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline/../StatisticsGen
!tree {PIPELINE_ROOT}/StatisticsGen

In [ ]:
# Visualize the outputted statistics of StatisticsGen
# context.show(statistics_gen.outputs['statistics'])

In [ ]:
# Access the artifact properties
for artifact in statistics_gen.outputs['statistics'].get():
    print(artifact)

<a name='5.2'></a><a id='5.2'></a>
## 5.2 SchemaGen
<a href="#top">[back to top]</a>

The SchemaGen component uses TensorFlow Data Validation to generate a 
schema from input statistics. The following TFX libraries use this schema:

* TensorFlow Data Validation
* TensorFlow Transform
* TensorFlow Model Analysis

Resources:

* https://www.tensorflow.org/tfx/guide/schemagen

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)

schema_gen.__dict__

In [ ]:
context.run(schema_gen, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline/../SchemaGen
!tree {PIPELINE_ROOT}/SchemaGen

In [ ]:
# Show the generated schema
# context.show(schema_gen.outputs['schema'])

<a name='5.3'></a><a id='5.3'></a>
## 5.3. ExampleValidator
<a href="#top">[back to top]</a>

The ExampleValidator pipeline component identifies anomalies in training and serving data. It can detect different classes of anomalies in the data.

Resources: 

* https://www.tensorflow.org/tfx/guide/exampleval

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

example_validator.__dict__

In [ ]:
context.run(example_validator, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline/../ExampleValidator
!tree {PIPELINE_ROOT}/ExampleValidator

In [ ]:
# context.show(example_validator.outputs['anomalies'])

In [ ]:
example_validator.with_platform_config

---
<a name='6.0'></a><a id='6.0'></a>
# 6. Data preprocessing
<a href="#top">[back to top]</a>

<a name='6.1'></a><a id='6.1'></a>
## 6.1 Transform
<a href="#top">[back to top]</a>

The Transform component wraps TensorFlow Transform (tf.Transform) to preprocess data in a TFX pipeline. This component will load the preprocessing_fn from input module file, preprocess both 'train' and 'eval' splits of input examples, generate the tf.Transform output, and save both transform function and transformed examples to orchestrator desired locations.

The Transform component can also invoke TFDV to compute statistics on the pre-transform and post-transform data. Invocations of TFDV take an optional StatsOptions object. To configure the StatsOptions object that is passed to TFDV for both pre-transform and post-transform statistics, users can define the optional stats_options_updater_fn within the module file.

* Resources:

https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Transform

---

**Model export and signature functions**:

1. Exporting the model.

When we export the model, we combine the preprocessing steps from the previous pipeline component with the trained model, and save the model in TensorFlow's SavedModel format. In the `run_fn` function, we define the model signature and save the model.

```python
signatures = {
    "serving_default": _get_serve_tf_examples_fn(
        model, tf_transform_output
    ).get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
    ),
}
```

* The `run_fn` exports the `get_serve_tf_examples_fn` as part of the model signature. 
* When a model has been exported and deployed, every prediction request will pass through the `serve_tf_examples_f()`, as shown here:

```python
# Applying the preprocessing graph to model inputs.
# This is called in the signature of run_fn().
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    # Load the preprocessing graph.
    # This layer is added as an attribute to the model in order to make sure that
    # the model assets are handled correctly when exporting.
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        
        # Note that the input is specified as tf.Example type, which comes from the client.
        # The client is not sending raw str, int, etc, but serialized tf.Example data.
        
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        # Parse the raw tf.Example records from the request.
        # Conversely, this implies the HTTP POST request must have its 
        # JSON payload in tf.Example format. 
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Apply the preprocessing transformation to raw data.
        transformed_features = model.tft_layer(parsed_features)
    
        # Perform prediction with preprocessed data.
        outputs = model(transformed_features)
        return {"outputs": outputs}

    return serve_tf_examples_fn
```

* With every request, we parse the serialized tf.Example records and apply the preprocessing steps to the raw request data. 
* The model then makes a prediction on the preprocessed data.


---

**Note that this export tf function creates a tf.Example dependency on the client.**

https://github.com/tensorflow/tfx/issues/1885



In [ ]:
# Transform and Model
_module_file = './module.py'

In [ ]:
%%writefile {_module_file}
# This file written from interactive_pipeline.ipynb

import os
from typing import Union
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_transform as tft

LABEL_KEY = "consumer_disputed"

################
# Transform code
################

# feature name, feature dimensionality
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}

# feature name, bucket count
BUCKET_FEATURES = {"zip_code": 10}

# feature name, value is unused
TEXT_FEATURES = {"consumer_complaint_narrative": None}


os.environ["TFHUB_CACHE_DIR"] = "tmp/tfhub"


def transformed_name(key: str) -> str:
    return key + "_xf"


def fill_in_missing(x: Union[tf.Tensor, tf.SparseTensor]) -> tf.Tensor:
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a
    dense tensor.

    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have
        size at most 1 in the second dimension.

    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    if isinstance(x, tf.sparse.SparseTensor):
        default_value = "" if x.dtype == tf.string else 0
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value,
        )
    return tf.squeeze(x, axis=1)


def convert_num_to_one_hot(label_tensor: tf.Tensor, num_labels: int = 2) -> tf.Tensor:
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def convert_zip_code(zipcode: str) -> tf.float32:
    """
    Convert a zipcode string to int64 representation. In the dataset the
    zipcodes are anonymized by repacing the last 3 digits to XXX. We are
    replacing those characters to 000 to simplify the bucketing later on.

    Args:
        str: zipcode
    Returns:
        zipcode: int64
    """
    zipcode = tf.strings.regex_replace(zipcode, r"X{0,5}", "0")
    zipcode = tf.strings.to_number(zipcode, out_type=tf.float32)
    return zipcode


def preprocessing_fn(inputs: tf.Tensor) -> tf.Tensor:
    """tf.transform's callback function for preprocessing inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        int_value = tft.compute_and_apply_vocabulary(
            fill_in_missing(inputs[key]), top_k=dim + 1
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            int_value, num_labels=dim + 1
        )

    for key, bucket_count in BUCKET_FEATURES.items():
        temp_feature = tft.bucketize(
            convert_zip_code(fill_in_missing(inputs[key])),
            bucket_count,
        )
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            temp_feature, num_labels=bucket_count + 1
        )

    for key in TEXT_FEATURES.keys():
        outputs[transformed_name(key)] = fill_in_missing(inputs[key])

    outputs[transformed_name(LABEL_KEY)] = fill_in_missing(inputs[LABEL_KEY])

    return outputs


################
# Model code
################


def get_model(show_summary: bool = True) -> tf.keras.models.Model:
    """
    This function defines a Keras model and returns the model as a Keras object.
    """

    # One-hot categorical features
    input_features = []
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    # Adding bucketized features
    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    # Adding text input features
    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,), name=transformed_name(key), dtype=tf.string)
        )


    # Embed text features
    MODULE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed = hub.KerasLayer(MODULE_URL)
    
    # Keras inputs are two-dimensional, but the encoder expects one-dimensional inputs.
    reshaped_narrative = tf.reshape(input_texts[0], [-1])
    embed_narrative = embed(reshaped_narrative)
    deep_ff = tf.keras.layers.Reshape((512,), input_shape=(1, 512))(embed_narrative)

    deep = tf.keras.layers.Dense(256, activation="relu")(deep_ff)
    deep = tf.keras.layers.Dense(64, activation="relu")(deep)
    deep = tf.keras.layers.Dense(16, activation="relu")(deep)

    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation="relu")(wide_ff)

    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation="sigmoid")(both)

    inputs = input_features + input_texts

    keras_model = tf.keras.models.Model(inputs, output)
    
    keras_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(),
        ],
    )
    if show_summary:
        keras_model.summary()

    return keras_model


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")








# Applying the preprocessing graph to model inputs.
# This is called in the signature of run_fn().
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    # Load the preprocessing graph.
    # This layer is added as an attribute to the model in order to make sure that
    # the model assets are handled correctly when exporting.
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        
        # Parse the raw tf.Example records from the request.
        # Conversely, this implies the HTTP POST request must have its 
        # JSON payload in tf.Example format. 
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Apply the preprocessing transformation to raw data.
        transformed_features = model.tft_layer(parsed_features)
    
        # Perform prediction with preprocessed data.
        outputs = model(transformed_features)
        return {"outputs": outputs}

    return serve_tf_examples_fn


# GB: batch_size=64 causes error at the smaller filesizes.
# def _input_fn(file_pattern, tf_transform_output, batch_size=64):
# https://github.com/tensorflow/models/issues/8487

# Changed to batch_size=16
def _input_fn(file_pattern, tf_transform_output, batch_size=16):

    """Generates features and label for tuning/training.
    Data loading for model training and validation is performed in batches, 
    and the loading is handled by the input_fn() function.

    The input_fn function lets us load the compressed, preprocessed datasets
    that were generated by the previous Transform step.
    To do this, we need to pass the tf_transform_output to the function. 
    This gives us the data schema to load the dataset from the TFRecord data 
    structures generated by the Transform component.
    By using the preprocessed datasets, we can avoid data preprocessing during 
    training and speed up the training process.
    The input_fn returns a generator (a batched_features_dataset) that will supply 
    data to the model one batch at a time.

    Args:
    file_pattern: input tfrecord file pattern.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

      Returns:
        A dataset that contains (features, indices) tuple where features is a
          dictionary of Tensors, and indices is a single Tensor of
          label indices.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # The dataset will be batched into the correct batch size.
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset



# TFX Trainer will call this function.
# The run_fn function receives a set of arguments, including the 
# transform graph, example datasets, and training parameters through 
# the fn_args object.

def run_fn(fn_args):
    """Train the model based on given args.

    The run_fn() function is a generic entry point to the training steps 
    and not tf.Keras specific. It carries out the following steps:
    
    1. Loading the training and validation data (or the data generator) 
    2. Defining the model architecture and compiling the model 
    3. Training the model
    4. Exporting the model to be evaluated in the next pipeline step

    This function is fairly generic and could be reused with any other 
    tf.Keras model. The project-specific details are defined in helper 
    functions like get_model() or input_fn().

    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """
    
    # Load the data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    # Call the input_fn to get data generators.
    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 64)
    
    
    
    # Call the get_model function to get the compiled Keras model.
    # strategy, mode = detect_platform()
    # print(strategy)
    # print(mode)
    
    # with strategy.scope():
    #     # Run compile step within distributed strategy
    model = get_model()

    

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
    callbacks = [tensorboard_callback]

    # Train the model using the number of training and evaluation steps passed by the Trainer component.
    model.fit(
        train_dataset,
        epochs=1,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
    )

    # Need to export the model.
    # Combine the preprocessing steps from the previous pipeline component with the trained model 
    # and save the model in TensorFlow’s SavedModel format.
    # Define a model signature based on the graph generated.
    # The run_fn exports the get_serve_tf_examples_fn as part of the model signature.
    # When a model has been exported and deployed, every prediction request will pass 
    # through serve_tf_examples_fn().
    # With every request, we parse the serialized tf.Example records and apply the 
    # preprocessing steps to the raw request data.
    # The model then makes a prediction on the preprocessed data.
    
    # Define the model signature, which includes the serving function we will describe later.
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    
    model.save(
        fn_args.serving_model_dir, 
        save_format="tf", 
        signatures=signatures
    )

<a name='6.1.1'></a><a id='6.1.1'></a>
## 6.1.1 tfx.components.Transform
<a href="#top">[back to top]</a>

The Transform component wraps TensorFlow Transform (tf.Transform) to 
preprocess data in a TFX pipeline.

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_module_file
)

transform.__dict__

In [ ]:
context.run(transform, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline/../Transform
!tree {PIPELINE_ROOT}/Transform

In [ ]:
transform.outputs

In [ ]:
# Examine the transform_graph artifact. It points to a directory containing 3 subdirs.
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

In [ ]:
# The transformed_metadata subdir contains the schema of the preprocessed data.
# The transform_fn subdir contains the actual preprocessing graph.
# The metadata subdir contains the schema of the original data.

# Take a look at some transformed examples.

# Get the URI of the output artifact representing the transformed examples, which is a directory.
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
print(train_uri)
HR()

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
print(tfrecord_filenames)
HR()

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
print(dataset)
HR()

# Iterate over the first n record(s) and decode them.
# for tfrecord in dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(tfrecord.numpy())
#     print(example)

In [ ]:
# A Dataset comprising records from one or more TFRecord files.
type(dataset)

---
<a name='7.0'></a><a id='7.0'></a>
# 7. Model training
<a href="#top">[back to top]</a>


<a name='7.1'></a><a id='7.1'></a>
## 7.1 Trainer
<a href="#top">[back to top]</a>

The Trainer component is used to train and eval a model using given inputs and a user-supplied run_fn function.

Resources

* https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Trainer



In [ ]:
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    for x in example.features.feature:
        print(x)  

In [ ]:
# for raw_record in dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)

In [ ]:
trainer_file = _module_file

In [ ]:
TRAINING_STEPS = 100 # 1000
EVALUATION_STEPS = 50

trainer = tfx.components.Trainer(
    module_file=trainer_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=tfx.proto.EvalArgs(num_steps=EVALUATION_STEPS)
    
    # tfx.proto.trainer_pb2 now deprecated
    # train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    # eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS)
)

trainer.__dict__

In [ ]:
context.run(trainer, enable_cache=False)

In [ ]:
!tree {PIPELINE_ROOT}/Trainer

### Load TensorBoard

To-do

---
<a name='8.0'></a><a id='8.0'></a>
# 8. Model analysis and validation
<a href="#top">[back to top]</a>


<a name='8.1'></a><a id='8.1'></a>
## 8.1 Resolver
<a href="#top">[back to top]</a>

Resolver is a special TFX node which handles special artifact resolution logics that will be used as inputs for downstream nodes.

To use Resolver, pass the followings to the Resolver constructor:

* Name of the Resolver instance
* A subclass of ResolverStrategy
* Configs that will be used to construct an instance of ResolverStrategy
* Channels to resolve with their tag, in the form of kwargs

Check:

**This component does not seem to create any artifacts on its own, but it usually used in combination with tfx.components.Evaluator, which does create artifacts.**

Resources:

* https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver

In [ ]:
# Resolver is a special TFX node which handles special artifact resolution 
# logics that will be used as inputs for downstream nodes.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(

          type=tfx.types.standard_artifacts.ModelBlessing
      ),
  # Note: .with_id() is a method of tfx.dsl.Resolver. 
  # This replaces the deprecated `instance_name` in TFX 0.30.0
).with_id('latest_blessed_model_resolver')

model_resolver.__dict__

In [ ]:
context.run(model_resolver, enable_cache=False)

In [ ]:
!tree {PIPELINE_ROOT}/Trainer

In [ ]:
model_resolver.id

<a name='8.2'></a><a id='8.2'></a>
## 8.2 Evaluator
<a href="#top">[back to top]</a>

A TFX component to evaluate models trained by a TFX Trainer component.

Resources:

* https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Evaluator

In [ ]:
# Note: This always blesses on first run even if below threshold

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="serving_default",
            label_key="consumer_disputed",
            # preprocessing_function_names=["transform_features"],
        )
    ],
    
    slicing_specs=[
        tfma.SlicingSpec(), 
        tfma.SlicingSpec(feature_keys=["product"])
    ],
    
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            # BinaryAccuracy must be over 0.45
                            lower_bound={"value": 0.45}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            # Validate if BinaryAccuracy is at least 0.01 higher than baseline model.
                            absolute={'value': 0.01} # -1e-10
                        )
                    ),
                ),
                tfma.MetricConfig(class_name="ExampleCount"),
                # tfma.MetricConfig(class_name="Precision"),
                # tfma.MetricConfig(class_name="Recall"),
                # tfma.MetricConfig(class_name="AUC"),
            ],
        )
    ],
)


In [ ]:
# The Evaluator TFX pipeline component performs deep analysis on the 
# training results for your models.
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)

evaluator.__dict__

In [ ]:
context.run(evaluator, enable_cache=False)

In [ ]:
# Check new artifacts in pipeline/../Evaluator
!tree {PIPELINE_ROOT}/Evaluator

In [ ]:
evaluator.outputs

In [ ]:
context.show(evaluator.outputs['evaluation'])

In [ ]:
# To see the visualization for sliced evaluation metrics, 
# directly call the TensorFlow Model Analysis library.

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# CURRENTLY BROKEN
# Show data sliced along feature column trip_start_hour.
# tfma.view.render_slicing_metrics(
#     tfma_result, slicing_column='trip_start_hour')

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

In [ ]:
# Also verify the success by loading the validation result record.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

<a name='8.3'></a><a id='8.3'></a>
## 8.3 Pusher
<a href="#top">[back to top]</a>

The Pusher component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to _serving_model_dir.


In [ ]:
# The Pusher component is used to push a validated model to a 
# deployment target during model training or re-training.

#from tfx.proto import pusher_pb2
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    )
)

pusher.__dict__

In [ ]:
context.run(pusher, enable_cache=False)

In [ ]:
# Examine the output artifacts of Pusher
pusher.outputs

In [ ]:
# The Pusher will export your model in the SavedModel format, which looks like this:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
print(push_uri)
HR()

model = tf.saved_model.load(push_uri)
print(model)
HR()

for item in model.signatures.items():
    #pp.pprint(item)
    print(item)

In [ ]:
!tree {SERVING_MODEL_DIR}

<a name='9.0'></a><a id='9.0'></a>
# 9. Testing SavedModels
<a href="#top">[back to top]</a>

In [192]:
import os
import glob

latest_model = max(glob.glob(os.path.join(SERVING_MODEL_DIR, '*/')), key=os.path.getmtime)
print("Latest model:\n")
print(latest_model)

Latest model:

/Users/gb/Desktop/TFX-CUSTOM-COMPONENTS/book/tfx_output_01/tfx_serving_model/customer_complaints/1667648432/


<a name='9.1'></a><a id='9.1'></a>
## 9.1 Testing inference with tf.keras.models.load_model
<a href="#top">[back to top]</a>

* Load the exported model and try some inferences with a few examples.
* After the initial loading, this is much faster then `saved_model_cli run`.
* Since we use the same payload format, this is also a better test for a HTTP client REST api call than `saved_model_cli run`.

In [201]:
print("Loading latest model")
loaded_model = tf.keras.models.load_model(latest_model)
HR()

inference_fn = loaded_model.signatures['serving_default']
print(type(inference_fn))

Loading latest model
----------------------------------------
<class 'tensorflow.python.saved_model.load._WrapperFunction'>


In [230]:
# Prepare an example and run inference.
# We can use example_bytes to also build up the payload the HTTP Request

features = {
    'zip_code':         tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'480XX'])),
    'state':            tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'CA'])),
    'company':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Encore Capital Group'])),
    'company_response': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Closed with explanation'])),
    'consumer_complaint_narrative': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'])),
    'issue':            tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Disclosure verification of debt'])),
    'product':          tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Debt collection'])),
    'timely_response':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'Yes'])),
}


# Creating a tf.Example standard proto storing data for training and inference.
# https://www.tensorflow.org/api_docs/python/tf/train/Example
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
print(f"example_proto type: {type(example_proto)}\n")
print("TF Example proto:\n")
print(example_proto)
HR()

    
# After serialization
# Serializes a proto message to a binary-string
# https://developers.google.com/protocol-buffers/docs/pythontutorial#parsing-and-serialization
example_bytes: bytes = example_proto.SerializeToString()
print(f"examples type: {type(example_bytes)}\n")
print("TF Example proto after serialization:\n")
print(example_bytes)
HR()



# The SavedModel target here (accessed directly) expects a Tensor in the examples argument.
# To enable this, we need to wrap it via `tf.constant([examples])`, 
# where tf.constant creates a constant tensor from a tensor-like object:
# https://www.tensorflow.org/api_docs/python/tf/constant

example_tensor = tf.constant([example_bytes]) # can also use tf.convert_to_tensor
print(f"example_tensor type: {type(example_tensor)}")
print()
print("example_tensor:\n")
print(example_tensor)
HR()


result = inference_fn(examples=example_tensor)
print(result)
HR()


print(result['outputs'].numpy())


example_proto type: <class 'tensorflow.core.example.example_pb2.Example'>

TF Example proto:

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Encore Capital Group"
      }
    }
  }
  feature {
    key: "company_response"
    value {
      bytes_list {
        value: "Closed with explanation"
      }
    }
  }
  feature {
    key: "consumer_complaint_narrative"
    value {
      bytes_list {
        value: "I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC."
      }
    }
  }
  feature {
    key: "issue"
    value {
      bytes_list {
        value: "Disclosure verification of debt"
      }
    }
  }
  feature {
 

<a name='9.2'></a><a id='9.2'></a>
## 9.2 saved_model_cli tool
<a href="#top">[back to top]</a>

Show and run computations on the MetaGraphDef in SavedModels.

In [202]:
print("Show all tag-sets in SavedModel:\n")
!saved_model_cli show --dir {latest_model}
HR()

print("Show all available SignatureDef keys in the MetaGraphDef specified by tag-set 'serve':\n")
!saved_model_cli show --dir {latest_model} --tag_set serve
HR()

print("Show all inputs and outputs TensorInfo for the specific SignatureDef 'serving_default' in the MetaGraph:")
!saved_model_cli show --dir {latest_model} --tag_set serve --signature_def serving_default

Show all tag-sets in SavedModel:

2022-11-07 11:00:32.484791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
The given SavedModel contains the following tag-sets:
'serve'
----------------------------------------
Show all available SignatureDef keys in the MetaGraphDef specified by tag-set 'serve':

2022-11-07 11:00:56.530411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
Signature

**Note**

This is the result of `saved_model_cli show --dir {latest_model} --tag_set serve --signature_def serving_default`:

```
The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['outputs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_5:0
Method name is: tensorflow/serving/predict
```

Note that shape: (-1) implies 1-D shape for inference data, eg {'examples': [{...}]}

This means the inference data needs to be in this format: 

```
examples=[{ 
    "zip_code": ["480XX"],
    "state": ["CA"],
    "company": ["Encore Capital Group"],
    "company_response": ["Closed with explanation"],
    "consumer_complaint_narrative": ["I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC."],
    "issue": ["Disclosure verification of debt"],
    "product": ["Debt collection"],
    "timely_response": [b"Yes"]
  }
```

<a name='9.2.1'></a><a id='9.2.1'></a>
### 9.2.1 Prediction test with saved_model_cli
<a href="#top">[back to top]</a>

The run command provides the following three ways to pass inputs to the model:

1. `--inputs`: pass numpy ndarray in files.
2. `--input_exprs`: pass Python expressions.
3. `--input_examples`: pass tf.train.Example.

Here we use the option `--input_examples`

**Important:**

With this option, we can pass tf.Examples data directly, without any manual serialization (which is required when passing data via a REST API to the TensorFlow Model). This lets us focus on getting the shape and keys of the payload correct first, w/o having to worry about format. 

https://www.tensorflow.org/guide/saved_model#run_command

#### Create command-line query

In [226]:
predict_cmd=f"""saved_model_cli run \
  --dir {latest_model} \
  --tag_set serve  \
  --signature_def serving_default  \
  --input_examples='examples=[{{ 
    "zip_code": ["480XX"],
    "state": ["CA"],
    "company": ["Encore Capital Group"],
    "company_response": ["Closed with explanation"],
    "consumer_complaint_narrative": ["I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC."],
    "issue": ["Disclosure verification of debt"],
    "product": ["Debt collection"],
    "timely_response": ["Yes"]
  }}]'
"""
#print(predict_cmd)

#### Run via command-line in notebook cell

In [53]:
# Run directly 
!{predict_cmd}

2022-11-07 08:58:59.259250: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 08:59:17.022488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /Users/gb/Desktop/TFX-CUS

---
<a name='10.0'></a><a id='10.0'></a>
# 10. Model deployment
<a href="#top">[back to top]</a>

<a name='10.1'></a><a id='10.1'></a>
## 10.1 TensorFlow ModelServer
<a href="#top">[back to top]</a>

Resources:

* https://github.com/tensorflow/serving/blob/master/tensorflow_serving/tools/docker/Dockerfile

In [235]:
model_name = 'my_model'
model_dir = SERVING_MODEL_DIR

log_file = f"{model_name}_tfs.log"
print(f"log_file: {log_file}")

log_file: my_model_tfs.log


<a name='10.1.1'></a><a id='10.1.1'></a>
### 10.1 Colab TF ModelServer
<a href="#top">[back to top]</a>

```
TensorFlow ModelServer: 2.8.0-rc1+dev.sha.9400ef1
TensorFlow Library: 2.8.0
```

In [ ]:
def tfs_download_colab():
    print("Running on Colab")
    HR()

    # tensorflow-model-server 2.8.2 and 2.9.0 require updated GLIBC versions.
    # https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/setup.md
    
    # Binary debian file
    debian_tfs = 'tensorflow-model-server-universal_2.8.0_all.deb'
    path = Path(debian_tfs)
    if not path.is_file():
        !wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'

    # Use dpkg to install, remove, and provide information about .deb packages.
    !dpkg -i tensorflow-model-server-universal_2.8.0_all.deb



def tfs_start_colab():
    # Use `subprocess.Popen()` to start the tensorflow_model_server. The subprocess module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.
    # To run `tensorflow_model_server` as a process on Debian, we additionaly need these steps:

    # * https://stackoverflow.com/questions/6011235/run-a-program-from-python-and-have-it-continue-to-run-after-the-script-is-kille/16928558#16928558
    # Run a program from python, and have it continue to run after the script is killed. If use 'setpgrp', this prevents the child from getting SIGHUP from the parent, since it's no longer part of the same process group, hence cannot use 'nohup',
    # but we really don't need it anymore.

    # * https://pythontect.com/python-subprocess-popen-tutorial/
    # If preexec_fn is set to a callable object, this object will be called in 
    # the child process just before the child is executed. (POSIX only)
    
    # By default, TensorFlow Model Server listens on port 8500 using the gRPC API. To use a different port, specify --port=<port number> on the command line.
    # By default TensorFlow Model Server will not listen for a REST/HTTP request. To specify a port to listen for a REST/HTTP request, include --rest_api_port=<port number> on the command line.

    # Can we directly pass environmental variables here?
    cmd = f"""nohup 
        tensorflow_model_server 
        --rest_api_port=8501 
        --model_name={model_name} 
        --model_base_path='{SERVING_MODEL_DIR}' 
        """

    try:
        # Run a program from python, and have it continue to run after the script is killed
        # https://stackoverflow.com/questions/6011235/run-a-program-from-python-and-have-it-continue-to-run-after-the-script-is-kille/16928558#16928558
        # If use 'setpgrp', this prevents the child from getting SIGHUP from the parent, 
        # since it's no longer part of the same process group, hence cannot use 'nohup',
        # but we really don't need it anymore.s
        # https://pythontect.com/python-subprocess-popen-tutorial/
        # If preexec_fn is set to a callable object, this object will be called in 
        # the child process just before the child is executed. (POSIX only)

        # popen on Unix is done using fork, so it naturally "detaches" from the parent process
        proc = subprocess.Popen(
            shlex.split(cmd),
            stdout=open(log_file, 'w'),
            stderr=subprocess.STDOUT,
            preexec_fn=os.setpgrp, # keep subprocess alive even when parent stops
            shell=False # needed for shell-like features like &
        )
        # print(f"-------- PROC HERE: {proc.pid}")
    except Exception as e:
        print(f"Error: {e}")

    return proc


def check_tfp_process():
    !ps ax | grep -e tensorflow_model_server


<a name='10.1.2'></a><a id='10.1.2'></a>
### 10.1.2 Docker TF ModelServer
<a href="#top">[back to top]</a>

* https://hub.docker.com/r/tensorflow/serving

Make sure to match the version running on Colab, which currently is `2.8.0-rc1-devel`

In [ ]:
cmd_cli = f"""
docker run \\
--rm --tty -p 8500:8500 -p 8501:8501 \\
--name {model_name} \\
--mount type=bind,source={model_dir},target=/models/{model_name} \\
--env MODEL_NAME={model_name} \\
--env TF_CPP_VMODULE='http_server=1' \\
--log-driver=json-file \\
--log-opt=mode=non-blocking \\
--detach \\
tensorflow/serving:2.8.0
"""

print(cmd_cli)
HR()


cmd_cli2 = f"""
docker run  \\
--rm --tty -p 8500:8500 -p 8501:8501 \\
-v "{model_dir}:/models/{model_name}" \\
--env MODEL_NAME={model_name} \\
--env TF_CPP_VMODULE='http_server=1' \\
--log-driver=json-file \\
--log-opt=mode=non-blocking \\
--detach \\
tensorflow/serving:2.8.0
"""
print(cmd_cli2)

# cmd_cli2 = f"""
# docker run -t --rm -p 8501:8501 \\
# -v {model_dir}:/models/{model_name} \\
# -e MODEL_NAME={model_name} \\
# --detach \\
# tensorflow/serving:2.8.0
# """

In [30]:
def server_docker():  
        
    cmd = f"""
docker run
--rm --tty -p 8500:8500 -p 8501:8501
--name {model_name}
--mount type=bind,source={model_dir},target=/models/{model_name}
--env MODEL_NAME={model_name}
--env TF_CPP_VMODULE='http_server=1'
--log-driver=json-file
--log-opt=mode=non-blocking
--detach
tensorflow/serving:2.8.0
"""
    
    print(cmd)
           
    try:
        proc = subprocess.Popen(
            shlex.split(cmd),
            stdout = subprocess.PIPE,
            stderr = subprocess.PIPE
        )
        
        # The communicate() method returns a tuple (stdoutdata, stderrdata)
        # It only reads data from stdout and stderr.
        out, err = proc.communicate()
        out = out.decode()
        err = err.decode()
        print(f"out: {(out.strip())}")
        
        if err:
            print(f"err: {err}")
                
        HR()
        
        sleep_time = 0.5 # Small time delay for docker instance to start up
        time.sleep(sleep_time)
        
    except subprocess.CalledProcessError as e:
        print(f"Subprocess error: {e.stderr}")
    except Exception as e:
        print(f"Error: {e}")
        
    return proc 

<a name='10.1.3'></a><a id='10.1.3'></a>
### 10.1.3 Start Server
<a href="#top">[back to top]</a>

In [31]:
if IN_COLAB:
    print("Running COLAB server")
    tfs_download_colab() # download
    proc = tfs_start_colab() # start
    !tensorflow_model_server --version
    print(f"tensorflow_model_server proc.pid: {proc.pid}")
    check_tfp_process() # check if running
else:
    print("Running Docker server")
    # Instantiate TFS with our model 
    proc = server_docker() 


Running Docker server

docker run
--rm --tty -p 8500:8500 -p 8501:8501
--name my_model
--mount type=bind,source=/Users/gb/Desktop/TFX-CUSTOM-COMPONENTS/book/tfx_output_01/tfx_serving_model/customer_complaints,target=/models/my_model
--env MODEL_NAME=my_model
--env TF_CPP_VMODULE='http_server=1'
--log-driver=json-file
--log-opt=mode=non-blocking
--detach
tensorflow/serving:2.8.0

out: bd22581262b0d1ff350f23dcbd53b2b19e51d54712f995871785ba4c1b496f58
----------------------------------------


In [ ]:
def show_tfs_dict(proc):
    print("Properties of returned container process:\n")
    for key in proc.__dict__:
        if key == 'args':
            HR()
            print(key, '->', proc.__dict__[key])
            HR()
        else:
            print(key, '->', proc.__dict__[key])

In [ ]:
show_tfs_dict(proc)

<a name='10.1.4'></a><a id='10.1.4'></a>
### 10.1.4 Check Server and Logs
<a href="#top">[back to top]</a>

In [ ]:
import time 

# Wait for the TFMS-docker server to spin up
time.sleep(5)

for x in range(10):    
    try:
        resp_data = requests.get(f'http://localhost:8501/v1/models/{model_name}')
        print(f"===> TFS Status : {resp_data.headers['Date']} {resp_data}")
        time.sleep(0.2)
    except Exception as e:
        print(f"Error: {e}")

HR()
    
!du -h {log_file}
HR()
!tail -10 {log_file}

In [ ]:
check_tfp_process() # check if running

In [ ]:
def request_status_rest():
    try:
        resp_data = requests.get(f'http://localhost:8501/v1/models/{model_name}')
    except Exception as e:
        print(f"Error: {e}")
    else:
        return resp_data
    
request_status_rest()

---
<a name="11.0"></a>
# 11. TensorFlow Model Server - RESTful APIs
<a href="#top">[back to top]</a>

Send requests with TensorFlow ModelServer RESTful APIs

<a name="11.1"></a>
## 11.1 Status API
<a href="#top">[back to top]</a>

* Returns the status of a model in the ModelServer.
* If successful, returns a JSON representation of `GetModelStatusResponse` protobuf.

In [47]:
try:
    resp_data = requests.get(f'http://localhost:8501/v1/models/{model_name}')
except Exception as e:
    print(f"Error: {e}")

pp.pprint(resp_data.json())

{   'model_version_status': [   {   'state': 'AVAILABLE',
                                    'status': {   'error_code': 'OK',
                                                  'error_message': ''},
                                    'version': '1667648432'}]}


<a name="11.2"></a>
## 11.2 Metadata API
<a href="#top">[back to top]</a>

* Returns the metadata of a model in the ModelServer. For example, we can inspect the details of the input and output tensors.
* Returns a JSON representation of `GetModelMetadataResponse` protobuf.

In [48]:
url = f"http://localhost:8501/v1/models/{model_name}/metadata"
print(url)
HR()

try:
    resp_data = requests.get(url)
except Exception as e:
    print(f"Error: {e}")

json_dump = (resp_data.json())
pp.pprint(repr(json_dump))

http://localhost:8501/v1/models/my_model/metadata
----------------------------------------
("{'model_spec': {'name': 'my_model', 'signature_name': '', 'version': "
 "'1667648432'}, 'metadata': {'signature_def': {'signature_def': "
 "{'serving_default': {'inputs': {'examples': {'dtype': 'DT_STRING', "
 "'tensor_shape': {'dim': [{'size': '-1', 'name': ''}], 'unknown_rank': "
 "False}, 'name': 'serving_default_examples:0'}}, 'outputs': {'outputs': "
 "{'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, "
 "{'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': "
 "'StatefulPartitionedCall_5:0'}}, 'method_name': "
 "'tensorflow/serving/predict'}, '__saved_model_init_op': {'inputs': {}, "
 "'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID', 'tensor_shape': "
 "{'dim': [], 'unknown_rank': True}, 'name': 'NoOp'}}, 'method_name': ''}}}}}")


<a name="11.3"></a>
## 11.3 Predict API (simple client)
<a href="#top">[back to top]</a>

* The request body for predict API must be JSON object formatted as follows (for clarity, it may be easier to always include the "signature_name" field)

* There are three base Feature types:
    - tf.train.BytesList
    - tf.train.FloatList
    - tf.train.Int64List

* Fundamentally, a tf.train.Example is a {"string": tf.train.Feature} mapping.

* TensorFlow Serving expects the input data as a JSON data structure.

* Create a Features message using tf.train.Example. The tf.train.Example message (or protobuf) is a flexible message type that represents a  {"string": value} mapping. It is designed for use with TensorFlow and is used throughout TFX.

Resources:

* https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft

---
### Client request for

If we look at this code in the saved model:

```python
def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        outputs = model(transformed_features)
        return {"outputs": outputs}
    return serve_tf_examples_fn
```

we can see that the input needs to be in serialized TF Example format. 

In particular, `tf.io.parse_example()` parses Example protos into a dict of tensors.
- https://www.tensorflow.org/api_docs/python/tf/io/parse_example

Hence this defines the format of data sent from our HTTP client. 

After preparing data as tf.Example, we then have to serialize it for transport via JSON. For this, we can use `json.dumps()`, which serializes obj to a JSON formatted str. Or, we can use `base64.b64encode`, which encodes the bytes-like objects using Base64 and returns the encoded bytes.

In [58]:
tf.train.Example.SerializeToString?

Docstring: Serializes the message to a string, only for initialized messages.
Type:      method_descriptor


In [236]:
# Python client for TensorFlow Model Server
# We need to manually serialize the tf.Example here
def client_simple(example_bytes):
    
    # The SavedModel target here (via TensorFlow Serving) expects serialized examples in json format.
    # To enable this, we use base64.b64encode(example_bytes).decode('utf-8')
    # If inputs is a scaler shape, may need to use this format:
    # "inputs": {"examples": {"b64": base64.b64encode(example_proto).decode('utf-8')}} 
    # Also note, since we use base64.b64encode here, we don't use json.dumps
    example_json = {
        "signature_name": "serving_default",
        "inputs": {"examples": [{"b64": base64.b64encode(example_bytes).decode('utf-8')}]} # 1-D format
    }

    print("Our payload, after base64 encoding and serialization:\n")
    print(example_json)
    HR()
    
    model_server_url = f"http://localhost:8501/v1/models/{model_name}:predict"
    headers = {"content-type": "application/json"}
    resp = requests.post(model_server_url, json=example_json, headers=headers)
    print("resp.json():")
    print(resp.json())
    HR()
    print("resp:")
    print(resp)

try:
    client_simple(example_bytes)
except Exception as e:
    print(f"Error: {e}")
    

Our payload, after base64 encoding and serialization:

{'signature_name': 'serving_default', 'inputs': {'examples': [{'b64': 'CpYFChoKD3RpbWVseV9yZXNwb25zZRIHCgUKA1llcwqrAwocY29uc3VtZXJfY29tcGxhaW50X25hcnJhdGl2ZRKKAwqHAwqEA0kgd2FzIGRlbmllZCBlbXBsb3ltZW50IGJlY2F1c2Ugb2YgYSBqdWRnbWVudCBhZ2FpbnN0IG1lLiBJIHdhcy8gYW0gY29tcGxldGVseSB1bmF3YXJlIG9mIGFueSBoZWFyaW5nLCBuZXZlciByZWNlaXZlZCBhbnkgbm90aWNlIG9mIGNvbGxlY3Rpb24gb2YgYSBkZWJ0IGJ5IE1pZGxhbmQgTExDIC4gTWlkbGFuZCBMTEMgYXBwYXJlbnRseSB0b29rIG1lIHRvIGNvdXJ0IHNvbWV3aGVyZSB3aXRob3V0IHNlcnZpbmcgbWUgYW55IGRvY3VtZW50cywgYW5kIHdvbiBhIGNvdXJ0cyBqdWRnZW1lbnQuIEkgd2FzIG5ldmVyIG5vdGlmaWVkIG9mIGFueSBoZWFyaW5nLCBjb21wbGFpbnQsIG9yIHJlY2VpdmVkIG5vdGljZSBvZiBjb2xsZWN0aW9uIG9mIGEgZGVidCBieSBNaWRsYW5kIExMQy4KIwoHY29tcGFueRIYChYKFEVuY29yZSBDYXBpdGFsIEdyb3VwCi8KEGNvbXBhbnlfcmVzcG9uc2USGwoZChdDbG9zZWQgd2l0aCBleHBsYW5hdGlvbgosCgVpc3N1ZRIjCiEKH0Rpc2Nsb3N1cmUgdmVyaWZpY2F0aW9uIG9mIGRlYnQKFQoIemlwX2NvZGUSCQoHCgU0ODBYWAoPCgVzdGF0ZRIGCgQKAkNBCh4KB3Byb2R1Y3QSEwoRCg9EZWJ0IG

<a name='11.4'></a><a id='11.4'></a>
## 11.4 Predict API (complex client)
<a href="#top">[back to top]</a>

* Create a tf.train.Example-compatible data type.
* This is a mostly-normalized data format for storing data for training and inference.
* This contains a key-value store features where each key (string) maps to a tf.train.Feature message
* Note that tf.train.Example message is just a wrapper around the Features message.

Resources: 

* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/saved_model_cli.py#L597
* https://www.tensorflow.org/api_docs/python/tf/train/Example
* https://www.tensorflow.org/tutorials/load_data/tfrecord
* https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft

In [239]:
def client_complex():

    def serialize_example(feature_dict):
        """Create a serialized tf.example from feature dictionary."""

        example = {}

        for feature_name, feature_list in feature_dict.items():
            if not isinstance(feature_list, list):
                raise ValueError(
                    f"feature value must be a list, but {feature_name}: {feature_list} is {type(feature_list)}"
                )

            # There are three base Feature types: tf.train.BytesList,tf.train.FloatList, tf.train.Int64List
            if isinstance(feature_list[0], float):
                example[feature_name] = (
                    tf.train.Feature(
                        float_list=tf.train.FloatList(value=[feature_list[0]])
                    ),
                )

            elif isinstance(feature_list[0], str):
                # feature_list[0] = feature_list[0].numpy()
                example[feature_name] = (
                    tf.train.Feature(
                        bytes_list=tf.train.BytesList(value=[feature_list[0]])
                    ),
                )

            elif isinstance(feature_list[0], bytes):
                # feature_list[0] = feature_list[0].numpy()
                example[feature_name] = (
                    tf.train.Feature(
                        bytes_list=tf.train.BytesList(value=[feature_list[0]])
                    ),
                )

            # https://www.tensorflow.org/tutorials/load_data/tfrecord
            # elif isinstance(feature_list[0], type(tf.constant(0))):
            #   feature_list[0] = feature_list[0].numpy() # BytesList won't unpack a string from an EagerTensor.
            #   example[feature_name] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[feature_list[0]])),

            elif isinstance(feature_list[0], int):
                example[feature_name] = (
                    tf.train.Feature(
                        int64_list=tf.train.Int64List(value=[feature_list[0]])
                    ),
                )

            else:
                raise ValueError(
                    f"Type {type(feature_list[0])} for value {feature_list[0]} is not supported in this operation"
                )

        # Create a Features message using tf.train.Example
        example_proto = tf.train.Example(features=tf.train.Features(feature=example))
        return example_proto.SerializeToString()


    # -----------------------------------------------

    # Use for testing:

    # Example: {"a": False, "b": 4, "c": b'goat', "d": 0.9876}

    feature_dict = {
        "zip_code": [b'480XX'],
        "state": [b'CA'],
        "company": [b'Encore Capital Group'],
        "company_response": [b'Closed with explanation'],
        "consumer_complaint_narrative": [b'I was denied employment because of a judgment against me. I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC . Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement. I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'],
        "issue": [b'Disclosure verification of debt'],
        "product": [b'Debt collection'],
        "timely_response": [b'Yes']
    }
    
    example_proto = serialize_example(feature_dict)

    # Testing: decode the message
    # example_proto_decoded = tf.train.Example.FromString(example_proto)
    # print("example_proto_decoded:\n")
    # print(example_proto_decoded)
    #HR()

    # -----------------------------------------------

    # Transform into JSON-compatible format
    # Apply base64.b64encode to base64 encode the bytes, then use decode('utf-8')
    # to create Base64 encoded data using human-readable characters.

    # Note: if 1-D format, may need to use this:
    # "inputs": {"examples": {"b64": base64.b64encode(example_proto).decode('utf-8')}} # scalar shape

    json_data = {
        "signature_name": "serving_default",
        "inputs": {
            "examples": [{"b64": base64.b64encode(example_proto).decode("utf-8")}]
        }  
    }

    print("JSON payload for request:\n")
    pp.pprint(json_data)
    HR()

    # -----------------------------------------------

    model_server_url = f"http://localhost:8501/v1/models/{model_name}:predict"
    headers = {"content-type": "application/json"}
    resp = requests.post(model_server_url, json=json_data, headers=headers)

    print(f"resp.json(): {resp.json()}")
    HR()
    print(f"resp: {resp}")
    
try:
    client_complex()
except Exception as e:
    print(f"Error: {e}")

JSON payload for request:

{   'inputs': {   'examples': [   {   'b64': 'CpYFChoKD3RpbWVseV9yZXNwb25zZRIHCgUKA1llcwojCgdjb21wYW55EhgKFgoURW5jb3JlIENhcGl0YWwgR3JvdXAKDwoFc3RhdGUSBgoECgJDQQqrAwocY29uc3VtZXJfY29tcGxhaW50X25hcnJhdGl2ZRKKAwqHAwqEA0kgd2FzIGRlbmllZCBlbXBsb3ltZW50IGJlY2F1c2Ugb2YgYSBqdWRnbWVudCBhZ2FpbnN0IG1lLiBJIHdhcy8gYW0gY29tcGxldGVseSB1bmF3YXJlIG9mIGFueSBoZWFyaW5nLCBuZXZlciByZWNlaXZlZCBhbnkgbm90aWNlIG9mIGNvbGxlY3Rpb24gb2YgYSBkZWJ0IGJ5IE1pZGxhbmQgTExDIC4gTWlkbGFuZCBMTEMgYXBwYXJlbnRseSB0b29rIG1lIHRvIGNvdXJ0IHNvbWV3aGVyZSB3aXRob3V0IHNlcnZpbmcgbWUgYW55IGRvY3VtZW50cywgYW5kIHdvbiBhIGNvdXJ0cyBqdWRnZW1lbnQuIEkgd2FzIG5ldmVyIG5vdGlmaWVkIG9mIGFueSBoZWFyaW5nLCBjb21wbGFpbnQsIG9yIHJlY2VpdmVkIG5vdGljZSBvZiBjb2xsZWN0aW9uIG9mIGEgZGVidCBieSBNaWRsYW5kIExMQy4KFQoIemlwX2NvZGUSCQoHCgU0ODBYWAovChBjb21wYW55X3Jlc3BvbnNlEhsKGQoXQ2xvc2VkIHdpdGggZXhwbGFuYXRpb24KLAoFaXNzdWUSIwohCh9EaXNjbG9zdXJlIHZlcmlmaWNhdGlvbiBvZiBkZWJ0Ch4KB3Byb2R1Y3QSEwoRCg9EZWJ0IGNvbGxlY3Rpb24='}]},
    'signature_name': 'serving_de

---
<a name='12.0'></a><a id='12.0'></a>
# 12. Miscellaneous
<a href="#top">[back to top]</a>

In [ ]:
# Check entire project structure
!tree -I __pycache__

In [ ]:
# Check total size of this project
!du -sch

In [ ]:
# Find biggest 10 folders/files at top level
!du -hs * | sort -rh | head -10

In [ ]:
# Find largest directories, inner level
!du -ha | sort -n -r | head -n 30

---
<a name="13.0"></a>
# 13. Quit TensorFlow Model Server 
<a href="#top">[back to top]</a>

In [ ]:
#!killall tensorflow_model_server -v

In [ ]:
# MIT License

# Copyright (c) 2020 Hannes Hapke
# Copyright (c) 2022 George Baptista

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
